## Hints to "How He Did That"
He wrote a lot about specifically what to do in [this book](https://drive.google.com/file/d/1l5324Nlqy_5r4y9UWtDCywMCCQLsZwfr/view?usp=sharing), (PDF pg. 6-8)

He also cites [this paper](https://drive.google.com/file/d/1REwUE5Uvv-ArSxuzUMIXj4EkB0-kovIs/view?usp=sharing) for how to implement the excitation function.

In [69]:
%matplotlib inline
from audiolazy import *
from librosa import load
from librosa.core.pitch import piptrack
import librosa
import matplotlib.pyplot as plt
import math
import numpy as np
import pretty_midi

# Bless https://ptolemy.berkeley.edu/eecs20/sidebars/hertz/index.html
def freq_to_rad(freq, sr):
    return (freq / sr) * (2 * pi)

def pulse_train(freq, sr, samples):
    period = 1 / freq # period in seconds
    dur = period * sr
    pulse_period = impulse(dur=dur)

    i = 0
    out = Stream([])
    while i < samples:
        if samples - i < dur:
            out.append(pulse_period.peek(samples - i))
            i = samples
        else:
            out.append(pulse_period.peek(dur))
            i += dur
            
    return out

def excitation(rad, samples):
    out = []
    
    if rad == 0.0:
        return white_noise(dur=samples).take(samples)
    
    N = int(pi // rad)
    
    d_i = 0
    for i in range(samples):
        denominator = sin_table[d_i]
        
        n_i = (((2 * N) + 1) * d_i) % DEFAULT_TABLE_SIZE
        numerator = sin_table[n_i]
        
        new_d_i = d_i + (((rad / 2) * DEFAULT_TABLE_SIZE) / (2 * pi))
        d_i = int(new_d_i) % DEFAULT_TABLE_SIZE
        
        if denominator == 0:
            out.append(1)
        else:
            out.append(numerator / denominator)
            
    return out

def get_fundamental(freqs, mags, t):
    if not t < freqs.shape[1]:
        return 0
    return freqs[mags[:, t].argmax(), t]

def miditofreq(midi): 
    return 2**((midi-69)/12.)*440

def mult(length, stretchFactor): 
    return int(length * stretchFactor)

def stretch(block, sr, stretchFactor):
    ori_arr = np.array(block)
    str_arr = librosa.core.resample(ori_arr, sr, sr * stretchFactor)
    return str_arr

## Analysis

In [70]:
analyses = {}

files = ["nuttiness.wav"]
BLK_SIZE = 441

def analyze_file(filename):
    sample = WavStream(filename)
    y, sr = load(filename, sr=sample.rate)

    freqs, mags = piptrack(y=y, sr=sr, fmin=50, fmax=300, hop_length=BLK_SIZE // 2)

    original = []
    coeffs = []
    resids = []
    resids_rms = []

    sample = list(sample)
    sample_len = len(sample) 
    num_blocks = 0
    i = 0

    while i < sample_len:
        if sample_len - i > BLK_SIZE:
            blk = sample[i:i + BLK_SIZE]
        else:
            blk = sample[i:]

        try:
            analysis_filt = lpc.ncovar(blk, 5)
        except:
            analysis_filt = lpc.covar(blk, 5)

        coeffs.append(analysis_filt)
        residual = analysis_filt(blk)
        resids.append(residual.peek(len(blk)))
        resids_rms.append(list(envelope(Stream(residual.peek(len(blk))))))
        synth_filt = 1 / analysis_filt
        amplified_blk = synth_filt(Stream(residual))
        original += amplified_blk.peek(len(blk))

        i += len(blk)
        num_blocks += 1
        
    analyses[filename] = {}
    analyses[filename]["original"] = original
    analyses[filename]["coeffs"] = coeffs
    analyses[filename]["resids"] = resids
    analyses[filename]["resids_rms"] = resids_rms
    analyses[filename]["num_blocks"] = num_blocks
    analyses[filename]["sr"] = sr
    
    with AudioIO(True) as player: # True means "wait for all sounds to stop"
        player.play(original, rate=sr, channels=2)
        
for file in files:
    analyze_file(file)

## Synthesis

In [109]:
def get_analysis(file_id):
    return analyses[file_id]["original"], analyses[file_id]["coeffs"], analyses[file_id]["resids"], analyses[file_id]["resids_rms"], analyses[file_id]["num_blocks"], analyses[file_id]["sr"]


def find_stretch_factor(file_id, noteLength, bpm):
    original, coeffs, resids, resids_rms, num_blocks, sr = get_analysis(file_id)
    return (noteLength * 4 * 60/bpm) * sr / len(original)

def make_note(f0, file_id, noteLength, bpm):
    stretchFactor = find_stretch_factor(file_id, noteLength, bpm)
    original, coeffs, resids, resids_rms, num_blocks, sr = get_analysis(file_id)
    new_f0 = miditofreq(f0)
    excit = Stream(excitation(freq_to_rad(new_f0, sr), mult(len(original), stretchFactor) ))
    final = []
    
    i = 0
    while i < num_blocks - 1:        
        filt = coeffs[i]
        synth_filt = 1 / filt
        excitation_pulse = excit.take(mult(BLK_SIZE, stretchFactor))
        stretched_resid = stretch(resids[i], sr, stretchFactor)
        excitation_pulse = [((0.05 * ex) + (0.95 * r)) for ex, r in zip(excitation_pulse, stretched_resid)]
        excitation_signal = [pulse * rms for pulse, rms in zip(excitation_pulse, stretch(resids_rms[i], sr, stretchFactor))]
        amplified_blk = list(synth_filt(excitation_signal))
        final += amplified_blk

        i += 1
        
    return final

    
def make_silence(noteLength, bpm, sr):
#     print(noteLength, bpm, sr)
    return [0] * int((noteLength * 4 * 60/bpm) * sr)

In [110]:
def get_tempo(pm_obj, t=0.0):
    """
    Gets the tempo of the MIDI file.
    :param t: time at which to get the tempo
    :return: the tempo of the MIDI file at time t in seconds per beat
    """
    tempo_change_times, tempi = pm_obj.get_tempo_changes()

    if t < 0:
        print("Unable to get tempo at time %f" % t)
        return

    for i in range(len(tempo_change_times)):
        tempo_change_time = tempo_change_times[i]
        tempo = tempi[i]

    # Look ahead for next tempo change
    if i < len(tempo_change_times) - 1:
        if t >= tempo_change_time and t < tempo_change_times[i + 1]:
            return 1 / (tempo / 60)
    # If this is the last tempo, return it
    else:
        return 1 / (tempo / 60)


def get_buffer_for_midi_file(midi_file_name, sound_file_id, bpm):
    original, coeffs, resids, resids_rms, num_blocks, sr = get_analysis(sound_file_id)
    midi_data = pretty_midi.PrettyMIDI(midi_file_name)
    tempo_scale = get_tempo(midi_data)
    track_buffer = []
    position = 0
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            new_start = note.start/tempo_scale
            new_end = note.end/tempo_scale
            if new_start > position:
                track_buffer += make_silence(new_start-position, bpm, sr)
            track_buffer += make_note(note.pitch, sound_file_id, new_end-new_start, bpm)
            position = new_end
    return track_buffer

trak=get_buffer_for_midi_file("testmidi.mid", "nuttiness.wav", 120)
                

## LENGTH CALCULATIONS ARE STILL a tad BUGGY

In [118]:
noteLen = 0.33
BPM = 120
file_id = "nuttiness.wav"
original, coeffs, resids, resids_rms, num_blocks, sr = get_analysis(file_id)
n_samp = sr * noteLen * 4 * 60/BPM
trak = make_note(60, file_id, noteLen, BPM)


print(len(original))
print(len(trak), n_samp, len(trak) - n_samp, (len(trak) - n_samp)%441, floor(n_samp/441)*441)
print(len(make_silence(noteLen, BPM, sr)), n_samp)
print(find_stretch_factor(file_id, noteLen, BPM), n_samp*1.0/len(original))

87394
(28908, 29106.0, -198.0, 243.0, 29106.0)
(29106, 29106.0)
(0.33304345836098587, 0.33304345836098587)


## Composition

In [12]:
def save(name, audio):
    librosa.output.write_wav(name, np.array(audio).reshape(-1, 2).transpose(), 44100)

